In [ ]:
import pandas as pd
import requests, json
import numpy as np
import time  
import pygsheets
from dotenv import load_dotenv
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('display.max_columns', None)  # Mostra todas as colunas
pd.set_option('display.width', None)        # Desativa quebra de linha automática
pd.set_option('display.expand_frame_repr', False)  # Mostra tudo em uma linha, se possível

In [ ]:
load_dotenv()  # Carrega as variáveis do .env

token = os.getenv("BEARER_TOKEN")
url_leads = os.getenv("URL_LEADS")
url_pipeline = os.getenv("URL_PIPELINE")

In [ ]:
headers = {
    "accept": "application/json",
    "authorization": f"Bearer {token}"
}

In [ ]:
response = requests.get(url_pipeline, headers=headers)

print(response.text)

In [ ]:
dados = response.json()['_embedded']['pipelines']

df1 = pd.DataFrame(dados)
df1.rename(columns={
    'id': 'id_pipeline',
    'name': 'pipeline'
}, inplace=True)
df1.head()

In [ ]:
# Cria nova coluna com os statuses
df1['statuses'] = df1['_embedded'].apply(lambda x: x.get('statuses', []))

# Expande a lista de statuses em linhas
df_statuses = df1.explode('statuses').reset_index(drop=True)

# Transforma os dicionários da coluna 'statuses' em colunas
df_statuses = pd.concat([df_statuses.drop(columns=['statuses', '_embedded']), df_statuses['statuses'].apply(pd.Series)], axis=1)


In [ ]:
df_statuses = df_statuses[['id','name','pipeline']].drop_duplicates(subset='id')
df_statuses.rename(columns={
"name": "step_name"
}, inplace=True)

In [ ]:
df_statuses

In [ ]:
all_leads = []
limit = 250
page = 1

while True:
    params = {
        "limit": limit,
        "page": page
    }
    
    response = requests.get(url_leads, headers=headers, params=params)
    
    if response.status_code != 200:
        print(f"Erro na página {page}: {response.status_code}")
        break

    data = response.json()['_embedded']['leads']
    
    if not data:
        break  # Não há mais leads
    
    all_leads.extend(data)
    print(f"Página {page} carregada: {len(data)} registros")
    
    if len(data) < limit:
        break  # Última página
    
    page += 1
    time.sleep(0.3)  # opcional: respeitar limites de rate

print(f"\nTotal de leads coletados: {len(all_leads)}")


In [ ]:
df = pd.DataFrame(all_leads)
df.head()

In [ ]:
# -------- 1. Expandir custom_fields_values --------
def extract_custom_fields(row):
    if isinstance(row, list):
        output = {}
        for item in row:
            field_name = item.get('field_name')
            values = item.get('values', [])
            if field_name and values:
                output[field_name] = values[0].get('value')
        return pd.Series(output)
    return pd.Series()

custom_fields_df = df['custom_fields_values'].apply(extract_custom_fields)

# Opcional: limpar nomes de colunas (substituir '/' por '_', etc.)
custom_fields_df.columns = [col.replace('/', '_').replace(' ', '_') for col in custom_fields_df.columns]

# -------- 2. Extrair nomes das tags --------
def extract_tags(row):
    if isinstance(row, list):
        return [tag.get('name') for tag in row if isinstance(tag, dict)]
    return []

df['tags'] = df['_embedded'].apply(extract_tags)

# # -------- 3. Extrair nome da empresa --------
# def extract_company_name(row):
#     if isinstance(row, list) and len(row) > 0:
#         return row[0].get('name')
#     return None

# df['company_name'] = df['_embedded.companies'].apply(extract_company_name)

# -------- 4. Concatenar tudo no DataFrame final --------
df = pd.concat([df, custom_fields_df], axis=1)

# (Opcional) Remover as colunas originais se desejar
#df.drop(columns=['custom_fields_values', '_embedded.tags', '_embedded.companies'], inplace=True)



In [ ]:
df.head(5)

In [ ]:
#df.to_csv('df_kommo_v3.csv', encoding="utf-8-sig")

In [ ]:
print(type(df['created_at'][0]))
print(type(df['updated_at'][0]))
print(type(df['closed_at'][0]))
print(type(df['closest_task_at'][0]))
print(type(df['Data_do_agendamento'].iloc[0]))
print(type(df['Data_de_criação'].iloc[0]))

Exploração


In [ ]:
""" df[['created_at', 
	   'updated_at', 
	   'closed_at', 
	   'closest_task_at',
       'Data_do_agendamento', 
       'Data_de_criação']] """

In [ ]:
# Lista das colunas com Unix Timestamps
unix_columns = [
    'created_at', 
    'updated_at', 
    'closed_at', 
    'closest_task_at',
    'Data_do_agendamento', 
    'Data_de_criação'
]

# Limite superior para timestamps válidos (01/01/2100)
max_valid_timestamp = 4102444800  # segundos desde 1970

# Conversão segura
for col in unix_columns:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')  # garante tipo numérico
        df[col] = df[col].where(df[col] < max_valid_timestamp, np.nan)  # remove absurdos
        df[col] = pd.to_datetime(df[col], unit='s', errors='coerce')  # converte para datetime


In [ ]:
df[['created_at', 
	   'updated_at', 
	   'closed_at', 
	   'closest_task_at',
       'Data_do_agendamento', 
       'Data_de_criação']]

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df_statuses.head()

In [ ]:
# Renomear colunas do mapeamento para evitar conflito
df_statuses = df_statuses.rename(columns={
    'id': 'status_id',       # para bater com o df_principal
    'step_name': 'status_name',
    'pipeline': 'pipeline_name'
})
df['status_id'] = df['status_id'].astype('Int64')  # Int64 com I maiúsculo permite valores NaN
df_statuses['status_id'] = df_statuses['status_id'].astype('Int64')

# Realizar o merge com base no status_id
df = df.merge(df_statuses, on='status_id', how='left')

In [ ]:
# Mapeamento dos IDs para nomes
mapa_vendedores = {
    "11188591": "Everton Oliveira",
    "13190131": "Vendedora Gabriele",
    "13190615": "Vendedor Daniel",
    "13190631": "Vendedor Leonardo"
}

# Converta a coluna responsible_user_id para string (se ainda não for)
df['responsible_user_id'] = df['responsible_user_id'].astype(str)

# Crie ou atualize a coluna 'vendedor' com base no dicionário
df['vendedor'] = df['responsible_user_id'].map(mapa_vendedores)


In [ ]:
#Tempo médio entre criação e agendamento
df['tempo_ate_agendamento'] = (df['Data_do_agendamento'] - df['Data_de_criação']).dt.days
#ciclo de venda (em dias)
df['ciclo_venda_dias'] = (df['closed_at'] - df['Data_de_criação']).dt.days


In [ ]:
df['agendou'] = df['Data_do_agendamento'].notna()


In [ ]:

df[df['status_name'] == 'Venda Concluída 🤑']


In [ ]:
gc = pygsheets.authorize(service_file='credenciais.json')

sh = gc.open('Dados Vendas Casa do Volante')
wks = sh.worksheet_by_title('kommo-api')

wks.clear()
wks.set_dataframe(df, (1, 1))  # Começa a escrever da célula A1


###
|Tipo de dado	Coluna|
|Dados do cliente|	id, name, responsible_user_id|     \
|Detalhes do serviço|	Serviço_que_entrou_em_contato, price|\
|Agendamento e datas|	Data_do_agendamento, Data_de_criação|\
|Localização|	Cidade_Bairro|\
|Funil de vendas|	status_id, pipeline_id, Motivo_de_perda|\
|Marketing|	Campanha, Conjunto, Nome_do_criativo|\
|Pagamento|	Método_de_pagamento|

In [ ]:
#Volume de Leads por período
df['Data_de_criação'].dt.to_period('M').value_counts().sort_index()


In [ ]:
#Percentual de leads que chegaram a agendar um horário
df['agendou'] = df['Data_do_agendamento'].notna()
agendados_pct = df['agendou'].mean()

In [ ]:
#Serviços mais procurados
df['Serviço_que_entrou_em_contato'].value_counts()

In [ ]:
#Receita estimada por serviço
df.groupby('Serviço_que_entrou_em_contato')['price'].sum().sort_values(ascending=False)


In [ ]:
#Motivos de perda mais frequentes
df['Motivo_de_perda'].value_counts()


In [ ]:
#Distribuição por localização
df['Cidade_Bairro'].value_counts()


In [ ]:
#Desempenho de campanhas
df['Campanha'].value_counts()
df.groupby('Campanha')['price'].sum()


In [ ]:
#Formas de pagamento preferidas
df['Método_de_pagamento'].value_counts()


In [ ]:


df['tempo_ate_agendamento'].mean()


In [ ]:
#Usuário responsável mais produtivo
df.groupby('responsible_user_id')['id'].count().sort_values(ascending=False)


In [ ]:
#Ciclo médio por usuário responsável
df.groupby('responsible_user_id')['ciclo_venda_dias'].mean().sort_values()


In [ ]:
df.sort_values(by='ciclo_venda_dias', ascending=False).head(10)


In [ ]:
df['ciclo_venda_dias'][0]

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
#quantos leads estão em cada etapa do funil: Conversão por Etapa (status_name)
df['status_name'].value_counts()


In [ ]:
# Conversão por Funil (pipeline_name)
df['pipeline_name'].value_counts()


In [ ]:
# Tempo até o agendamento por etapa (ciclo_venda_dias ou tempo_ate_agendamento)
df.groupby('status_name')['ciclo_venda_dias'].mean().sort_values()


In [ ]:
# Taxa de agendamento por etapa ou funil
df.groupby('status_name')['agendou'].mean().sort_values(ascending=False)
df.groupby('pipeline_name')['agendou'].mean()


In [ ]:
#  Funis com mais perdas (Motivo_de_perda) por pipeline
df[df['agendou'] == False].groupby('pipeline_name')['Motivo_de_perda'].value_counts()


In [ ]:
#Conversão por etapa até venda (status_id == 142)
df[df['status_name'] == 'Venda Concluída 🤑'].groupby('pipeline_name').size()


In [ ]:
#Distribuição de preço por status
df.groupby('status_name')['price'].describe()


In [ ]:
# Filtrar etapas críticas com tempo alto e conversão baixa:
df.groupby('status_name').agg({
    'ciclo_venda_dias': 'mean',
    'agendou': 'mean',
    'id': 'count'
}).sort_values(by='ciclo_venda_dias', ascending=False)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(data=df, y='status_name', order=df['status_name'].value_counts().index)
plt.title('Distribuição de Leads por Etapa do Funil')
plt.xlabel('Quantidade de Leads')
plt.ylabel('Etapa')
plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.countplot(data=df, y='status_name', order=df['status_name'].value_counts().index)
plt.title('Quantidade de Leads por Etapa')
plt.xlabel('Leads')
plt.ylabel('Etapa')
plt.tight_layout()
plt.show()


In [ ]:
taxa = df.groupby('status_name')['agendou'].mean().sort_values()
taxa.plot(kind='barh', figsize=(10, 6), title='Taxa de Conversão por Etapa (%)')
plt.xlabel('% de Leads Agendados')
plt.tight_layout()
plt.show()


In [ ]:
df['pipeline_name'].value_counts().plot(kind='pie', autopct='%1.1f%%', figsize=(6, 6), title='Distribuição de Leads por Funil')
plt.ylabel('')
plt.tight_layout()
plt.show()
